In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import PIL
import PIL.Image

In [20]:
count = 0

cleaned_training_set = './cleaned/Training/'
cleaned_testing_set = './cleaned/Testing'
for filename in os.listdir(cleaned_testing_set):
        filepath = os.path.join(cleaned_training_set, filename)
        for file in os.listdir(filepath):
            full_file_path = os.path.join(filepath, file)  # {{ edit_1 }}
            if os.path.isfile(full_file_path):
                print(f"adding {file} to the count")
                count += 1
            else:
                print(f"{file} is not a file")
                

print(f"The total number of files from {cleaned_testing_set} is {count}")


    

adding Tr-pi_0505.jpg to the count
adding Tr-pi_0263.jpg to the count
adding Tr-pi_0277.jpg to the count
adding Tr-pi_1169.jpg to the count
adding Tr-pi_0511.jpg to the count
adding Tr-pi_0539.jpg to the count
adding Tr-pi_1141.jpg to the count
adding Tr-pi_1155.jpg to the count
adding Tr-pi_1182.jpg to the count
adding Tr-pi_1196.jpg to the count
adding Tr-pi_0288.jpg to the count
adding Tr-pi_0061.jpg to the count
adding Tr-pi_0707.jpg to the count
adding Tr-pi_1419.jpg to the count
adding Tr-pi_0713.jpg to the count
adding Tr-pi_0075.jpg to the count
adding Tr-pi_1343.jpg to the count
adding Tr-pi_1425.jpg to the count
adding Tr-pi_1431.jpg to the count
adding Tr-pi_1357.jpg to the count
adding Tr-pi_0049.jpg to the count
adding Tr-pi_0908.jpg to the count
adding Tr-pi_1380.jpg to the count
adding Tr-pi_0920.jpg to the count
adding Tr-pi_0934.jpg to the count
adding Tr-pi_1394.jpg to the count
adding Tr-pi_0115.jpg to the count
adding Tr-pi_0673.jpg to the count
adding Tr-pi_0667.jp

In [23]:
def get_image_directory(dir_path, directory):
    
    

['.', '/', 'c', 'l', 'e', 'a', 'n', 'e', 'd', '/', 'T', 'r', 'a', 'i', 'n', 'i', 'n', 'g', '/', 'g', 'l', 'i', 'o', 'm', 'a']
